### 성능지표 - 분류
- 정확도 (Accuracy) : 전체 정답에서 맞춘 정답의 수
- 정밀도 (Precision) : 모델(예측) 기준으로 모델이 True라고 한 것이 정답(실제) True인 것
- 재현율 (Recall) : 정답(실제) 기준으로 정답(실제)이 True라고 한 것이 모델(예측) True인 것
- F1-score : 재현율와 정밀도 2가지를 조합한 점수
- 오차행렬(Counfusion Matrix) : 정답과 예측값의 관계 나타내는 표

[1] 모듈 로딩 및 데이터 준비

In [47]:
# 모듈 로딩
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score          # sklearn에서 제공하는 모델 성능평가 관련 모듈

import pandas as pd
import numpy as np


In [48]:
# 데이터 로딩
digitBunch = load_digits(as_frame=True)

[2] 피쳐와 레이블 추출

In [49]:
# 0~9까지 이미지 데이터 값
imgDF = digitBunch['data']

# 0~9 라벨
labelSR = digitBunch['target']
labelSR.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [50]:
## => 이진분류를 위해서 7과 나머지 라벨 변경
mask = labelSR == 7
labelSR[~mask] # 7 아닌 애들만 True


0       0
1       1
2       2
3       3
4       4
       ..
1792    9
1793    0
1794    8
1795    9
1796    8
Name: target, Length: 1618, dtype: int32

In [51]:
labelSR[~mask] = 0
labelSR[mask] = 1

In [52]:
labelSR.value_counts()

target
0    1618
1     179
Name: count, dtype: int64

[3] 학습용 데이터셋 준비

In [53]:
X_train,X_test,y_train,y_test = train_test_split(imgDF,labelSR,
                                                 stratify = labelSR,
                                                 random_state=1)

In [54]:
y_train.value_counts()

target
0    1213
1     134
Name: count, dtype: int64

In [55]:
print('[Train]',y_train.value_counts()/y_train.shape[0])
print('[Test]',y_test.value_counts()/y_test.shape[0])

[Train] target
0    0.90052
1    0.09948
Name: count, dtype: float64
[Test] target
0    0.9
1    0.1
Name: count, dtype: float64


[4] 클래스 정의

In [56]:
# 사용자 정의 클래스 DummyClass : 아무것도 하지 않는 클래스, 단순 테스트용
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):

    def fit(self,X,y):pass

    def predict(self,X) : return np.zeros((len(X),1),dtype=bool)

[5] 학습

In [57]:
# 인스턴스 생성
model = MyClassifier()

# 학습
model.fit(X_train,y_train)

[6] 예측 및 성능 평가

In [58]:
pre_ = model.predict(X_train)
pre_

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

[6-1] 정확도 => 불균형 데이터 경우 신뢰 불가!!!

In [59]:
accuracy_score(y_train,pre_)
# 불균형 데이터 상태에서는 accuracy를 믿을게 못됨.

0.9005196733481812

[6-2] 오차행렬

In [60]:
from sklearn.metrics import confusion_matrix, classification_report

In [61]:
y_test = [0,0,0,1,1,    1,0,0,0,1,    1,1,1,0,0,    0,0,0,0,0]
y_pre = [1,1,1,1,1, 1,0,1,0,1, 1,1,1,0,0, 0,0,0,0,0]

In [62]:
# 정답과 예측값 전달
tn, fp, fn, tp= confusion_matrix(y_test,y_pre).reshape(-1)

In [63]:
print(tn, fp, fn, tp)

9 4 0 7


- 정밀도 (Precision) : 모델(예측) 중심!!

In [68]:
from sklearn.metrics import precision_score

precision_score(y_test,y_pre)

0.6363636363636364

* 재현율(Recall) : 정답(실제) 중심!!

In [70]:
from sklearn.metrics import recall_score

recall_score(y_test,y_pre)

1.0

데이터가 불균형 데이터일 경우 accuracy를 믿지 못함. => 데이터 불균형을 조절해주거나 다른 지표들을 함께 봐야됨.